Hadoop en modo standalone (sin hdfs)
===

* *30 min* | Última modificación: Noviembre 14, 2019

## Definición del problema

Se desea contar la frecuencia de ocurrencia de palabras en un conjunto de documentos. Debido a los requerimientos de diseño (gran volúmen de datos y tiempos rápidos de respuesta) se desea implementar una arquitectura Big Data. 


Se desea **probar** la implementación en **Python** y ejecutar Hadoop en **modo standalone** para depuración.

## Solución

A continuación se usa la opción streaming de Hadoop. Se el tutorial anterior como punto de inicio.

### Preparación

#### Inicio de la máquina virtual

Si usa linux o macOS puede pasar directamente al siguiente paso. Inicie la VM con:

```bash
vagrant up
```

y luego vaya a la carpeta de trabajo:

```
cd /vagrant
```


#### Ejecución del contendor de Docker

Si va a iniciar el contendor de Hadoop en la carpeta compartida con su máquina local use:

```
docker run --rm -it -v "$PWD":/datalake --name hadoop -p 8888:8888 jdvelasq/hadoop:2.8.5-standalone
```

Si desea iniciar la sesión en el `datalake` use:

```
docker run --rm -it -v datalake:/datalake --name hadoop -p 8888:8888 jdvelasq/hadoop:2.8.5-standalone
```


Si un contenedor ya se está ejecutando puede abrir un nuevo terminal con:

```
docker exec -it hadoop bash
```

#### Archivos de prueba

A continuación se generarán tres archivos de prueba para probar el sistema. Puede usar directamente comandos del sistema operativo en el Terminal y el editor de texto `pico` para crear los archivos.

In [1]:
## Se crea el directorio de entrada en la carpeta actual simulando el HDFS
!hadoop fs -mkdir input

In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing input/text2.txt


### Prueba de la implementación dentro de Hadoop

En el modo standalone, el hdfs opera directamente sobre los archivos de la máquina local. En este caso en la máquina virtual creada Vagrant.

La implementación es idéntica y solo cambia el manejo de archivos.

#### Paso 1

In [5]:
%%writefile mapper.py
##
## Esta es la funcion que mapea la entrada a parejas (clave, valor)
##
import sys
if __name__ == "__main__": 
    
    ##
    ## itera sobre cada linea de codigo recibida
    ## a traves del flujo de entrada
    ##
    for line in sys.stdin:
        
        ##
        ## genera las tuplas palabra \tabulador 1
        ## ya que es un conteo de palabras
        ##
        for word in line.split(): 
                   
            ##
            ## escribe al flujo estandar de salida
            ##
            sys.stdout.write("{}\t1\n".format(word))
            

Writing mapper.py


#### Paso 2

In [6]:
## la salida de la función anterior es:
!cat /tmp/input/text*.txt | python3 mapper.py | head

cat: '/tmp/input/text*.txt': No such file or directory


### Paso 3

Se implementa la función `reduce` y se salva en el archivo `reducer.py`.

In [7]:
%%writefile reducer.py

import sys

##
## Esta funcion reduce los elementos que 
## tienen la misma clave
##

if __name__ == '__main__': 
  
    curkey = None
    total = 0
    
    ##
    ## cada linea de texto recibida es una 
    ## entrada clave \tabulador valor
    ##
    for line in sys.stdin:
        
        key, val = line.split("\t") 
        val = int(val)
        
        if key == curkey: 
            ##
            ## No se ha cambiado de clave. Aca se 
            ## acumulan los valores para la misma clave.
            ##
            total += val  
        else:
            ##
            ## Se cambio de clave. Se reinicia el
            ## acumulador.
            ##
            if curkey is not None:
                ##
                ## una vez se han reducido todos los elementos
                ## con la misma clave se imprime el resultado en
                ## el flujo de salida
                ##
                sys.stdout.write("{}\t{}\n".format(curkey, total)) 
            
            curkey = key
            total = val
            
    sys.stdout.write("{}\t{}\n".format(curkey, total)) 

Writing reducer.py


#### Paso 3

In [8]:
##
## La función sort hace que todos los elementos con 
## la misma clave queden en lineas consecutivas.
## Hace el papel del módulo Shuffle & Sort
##
!cat ./input/text*.txt | python3 mapper.py | sort | python3 reducer.py | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


### Ejecución en Hadoop en modo standalone

#### Paso 1. Movimiento de los datos al HDFS

Se copian los archivos de entrada del sistema local (o del datalake) al HDFS.

In [9]:
!hadoop fs -mkdir tmp
!hadoop fs -mkdir tmp/input
!hadoop fs -copyFromLocal input/* tmp/input
!hadoop fs -ls tmp/input/*

-rw-r--r--   1 root root       1093 2019-11-15 02:17 tmp/input/text0.txt
-rw-r--r--   1 root root        352 2019-11-15 02:17 tmp/input/text1.txt
-rw-r--r--   1 root root        440 2019-11-15 02:17 tmp/input/text2.txt


Todos los comandos de la opción fs de Hadoop siguen siendo válidos.

#### Paso 2

In [10]:
##
## Se ejecuta en Hadoop.
##   -files: archivos a copiar al hdfs
##   -input: archivo de entrada
##   -output: directorio de salida
##   -file: archivos a copiar de la maquina local al hdfs
##   -maper: programa que ejecuta el map
##   -reducer: programa que ejecuta la reducción
##
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar -input tmp/input  -output tmp/output -mapper 'python3 mapper.py' -reducer 'python3 reducer.py'

In [11]:
## contenido del directorio con los 
## resultados de la corrida
!hadoop fs -ls tmp/output

Found 2 items
-rw-r--r--   1 root root          0 2019-11-15 02:17 tmp/output/_SUCCESS
-rw-r--r--   1 root root       1649 2019-11-15 02:17 tmp/output/part-00000


In [12]:
## se visualiza el archivo con los
## resultados de la corrida
!hadoop fs -cat tmp/output/part-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


### Limpieza de la máquina local

In [13]:
!rm reducer.py mapper.py
!rm -rf input tmp